In [1]:
from collections import deque, defaultdict

def solution_1(inputs):
    rows, cols = len(inputs), len(inputs[0])
    
    def backtracking(x, y, path):
        nonlocal seen
        if (x, y) == (rows-1, cols-1):
            res.append(path[::])
            return # 走出去了
            
        seen.add((x, y))
        for nx, ny in (x+1, y), (x-1, y), (x, y+1), (x, y-1):
            # 走出边界了，不是连通的区域
            if nx < 0 or ny < 0 or nx >= rows or ny >= cols:
                continue
            if inputs[nx][ny] == 1 or (nx, ny) in seen:
                continue
            path.append((x, y))
            backtracking(nx, ny, path)
            path.pop()
        return
    
    res = []
    seen = set()
    for i in range(rows):
        for j in range(cols):
            if inputs[i][j] == 1 or (i, j) in seen:
                continue
            if (i, j) == (rows-1, cols-1):
                continue
            backtracking(i, j, [])
    return res

In [2]:
solution_1([[0, 0, 0, 0, 0], 
            [0, 1, 1, 1, 0], 
            [0, 1, 0, 1, 0], 
            [0, 1, 1, 1, 0], 
            [0, 0, 0, 0, 0]])

[[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)],
 [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (1, 4), (2, 4), (3, 4)]]

In [ ]:
def solution_2(inputs):
    
    def find(x):
        if father[x] != x:
            father[x] = find(father[x])
        return father[x]
    
    def union(x, y):
        x = find(x)
        y = find(y)
        if x != y:
            father[x] = y
        
    def dfs(x, y):
        nonlocal seen
        nodes = deque()
        nodes.append((x, y))
        seen.add((x, y))
        root  = cols * x + y # 横纵坐标转换为单值的形式
        while nodes:
            for _ in range(len(nodes)):
                i, j = nodes.popleft()
                for ni, nj in (i+1, j), (i-1, j), (i, j+1), (i, j-1):
                    # 走出边界了，不是连通的区域
                    if ni < 0 or nj < 0 or ni >= rows or nj >= cols:
                        return False
                    if (ni, nj) in seen or inputs[ni][nj] == 0: # 已经看过了，pass
                        continue
                    
                    idx = cols * ni + nj
                    # 不是一个连通域，union一起
                    if find(root) != find(idx):
                        union(root, idx)
                    seen.add((ni, nj))
                    nodes.append((ni, nj))
        return
    
    # 使用 union fold 的方式
    rows, cols = len(inputs), len(inputs[0])
    father = [x for x in range(rows * cols)] 
    seen = set() # 已经看过的地方记录
    for i in range(rows):
        for j in range(cols):
            if inputs[i][j] == 0:
                continue
            if (i, j) in seen:
                continue
            dfs(i, j)
    
    # 将属于同一个连通域的整合，同一个连通域，father相同
    res = defaultdict(list)
    for i in range(rows):
        for j in range(cols):
            if inputs[i][j] == 0:
                continue
            f = find(j * cols + i) # 寻找 father
            res[f].append((i, j))
    return res


def solution_3(inputs):
    
    def is_hole(r, c):
        # 判断当前位置是不是空洞
        nodes = deque()
        nodes.append((r, c))
        seen = {(r, c)}
        # 如果从当前位置，只走 val = 0 的坐标位置，可以走出图像的边缘，说明不是孔洞
        while nodes:
            for _ in range(len(nodes)):
                x, y = nodes.popleft()
                # 达到了图像的边界位置，不是孔洞
                if x == 0 or y == 0 or x == rows-1 or y == cols-1:
                    return False
                # 向四个方向走
                for nx, ny in (x+1, y), (x-1, y), (x, y+1), (x, y-1):
                    if inputs[nx][ny] == 1:
                        continue
                    if (nx, ny) in seen:
                        continue 
                    if inputs[x][y] == 0: # 只能走 val = 0的位置
                        seen.add((x, y))
                        nodes.append((x, y))
        return True
    
    rows, cols = len(inputs), len(inputs[0])
    res  = [] # 输出的坐标 是图像矩阵中的孔洞
    for i in range(rows):
        for j in range(cols):
            if inputs[i][j] == 1:
                continue
            if is_hole(i, j):
                res.append((i, j))
                
    # 填充孔洞位置
    for x, y in res:
        inputs[x][y] = 1
    return inputs

In [ ]:
    
    def is_connect(x, y):
        nodes = deque()
        nodes.append((x, y))
        seen = {(x, y)}
        cnt = 0 # 走了多少步
        while nodes:
            for _ in range(len(nodes)):
                i, j = seen.popleft()
                
                for ni, nj in (i+1, j), (i-1, j), (i, j+1), (i, j-1):
                    # 走出边界了，不是连通的区域
                    if ni < 0 or nj < 0 or ni >= rows or nj >= cols:
                        return False
                    if (ni, nj) in seen: # 已经看过了，pass
                        continue
                    if inputs[ni][nj] == 0:
                        continue
                    if cnt > 1 and (ni, nj) in seen:
                        pass
                    seen.add((x, y))
                    nodes.append((x, y))
            cnt += 1
        return
    
    # 所有 1 连通区域采用左边的形式记录
    res = []
    connected = set() # 已经联通的点
    for i in range(rows):
        for j in range(cols):
            if inputs[i][j] == 0:
                continue
            # 当前点已经联通了，pass
            if (i, j) in connected:
                continue
            state, nodes = is_connect(i, j)
            if state:
                # 如果是联通的，记录
                res.append(nodes)
    return res